In [1]:
from src.signal_generator import SignalGenerator
from src.metrics import Metric
from src.data_type import Measurement
import matplotlib.pyplot as plt
from src.data_compressor.other import CompressNTHS
from src.data_compressor.other import CompressMinMax
from src.data_compressor.other import CompressPWP
from src.data_compressor.pip import CompressPIP_ED
from src.data_compressor.pip import CompressPIP_PD
from src.data_compressor.pip import CompressPIP_VD
from src.data_compressor.paa import CompressPAA
from src.data_compressor.paa import CompressPAAVI
from src.data_compressor.paa import CompressByChunk
from src.data_compressor.pla import CompressAPCADFT
from src.data_compressor.pla import CompressAPCAFFT
from src.data_compressor.pla import CompressSTC
from src.data_compressor.pla import CompressHigherDeriveration
from src.data_type import Measurement
from src.data_compressor.compressor import Compressor
from typing import List, Dict
from collections import defaultdict as dict

from src.method_selector import AlgorythmSelector

In [3]:
import matplotlib.pyplot as plt
from typing import List
def vizualize(original_data, compressed_data, show_compressed: bool = True, show_interpolation: bool = True):
  x_original = [measurement.timestamp for measurement in original_data]
  y_original = [measurement.value for measurement in original_data]
  plt.figure(dpi=150)
  if show_interpolation:
      plt.plot(x_original, y_original, 'b')
  plt.plot(x_original, y_original, 'bo')
  if show_compressed:
    x_compressed = [measurement.timestamp for measurement in compressed_data]
    y_compressed = [measurement.value for measurement in compressed_data]
    if show_interpolation:
      plt.plot(x_compressed, y_compressed, 'r')
    plt.plot(x_compressed, y_compressed, 'ro')
  plt.grid()
  plt.show()

# vizualize(data, self.metrics_containter._interpolate_data(data, compressed_data))

In [ ]:
signal_generator = SignalGenerator(0, 100).with_peaks(3).with_peaks(3, direction=-1).sin(0.2, 0.2)
measurements = [Measurement(measurement, index * 100) for index, measurement in enumerate(signal_generator.data)]

In [4]:
AlgorythmSelector().get_best(measurements)